In [2]:
import pandas as pd

![Probabilities](ethicalmldiagram.png)

# P(race) (this would be given)

In [13]:
ProbRace = pd.read_csv('ProbRace.csv')[['Demographic','Percentage']]
ProbRace.head()

,Demographic,Percentage
0,white,0.765109
1,black,0.104995
2,hispanic,0.084471
3,asian,0.045425


# P(good or bad | race)  (this would be given, it is the pi values)

In [14]:
ProbGoodGivenRace = pd.read_csv("ProbGoodGivenRace.csv")
ProbGoodGivenRace

,white,black,hispanic,asian
0,0.759185,0.315164,0.550595,0.80066


# P(score>=x | good, race) (this they would calculate)

In [17]:
ProbScoreGreaterThanXGivenGoodAndRace = pd.read_csv('ProbScoreGreaterThanXGivenGoodAndRace.csv')
ProbScoreGreaterThanXGivenGoodAndRace.head()

,TransRisk Score,asian,black,hispanic,white
0,0.0,0.000000,0.000000,0.000000,0.000000
1,0.5,0.000000,0.000161,0.000172,0.000000
2,1.0,0.000568,0.003032,0.000527,0.000304
3,1.5,0.000568,0.003701,0.000527,0.000401
4,2.0,0.000895,0.004625,0.000874,0.000606


<hr/>

# How they will calculate P(score>=x | good, race) for tutorial

## First: find P(score = x | good, race)
### P(score=x | good, race) = P(score=x & good & race) / P(good and race)
<hr/>
### Step 1:
### P(score=x & good & race ) = P(race & score=x) * P(good | race, score=x)
###  P(score=x & good & race ) = P(race) * P(score=x | race) * P(good | race, score=x)
<hr/>
### Step 2:
### P(good and race) = P(race) * P(good | race)
<hr/>
### Step 3:
### Step 1 / Step2

## What they will be given

** ProbRaceAndScoreEqualsX: P(race & score=x) **

** ProbGoodGivenRaceAndScoreEqualsX : P(good | race, score=x) **

** ProbRace : P(race) **

** ProbScoreEqualsXGivenRace : P(score=x | race) **

** ProbGoodGivenRaceAndScoreEqualsX : P(good | race, score=x)**

** ProbGoodGivenRace : P(good | race) **

##  P(score>=x | good, race) = P(score = x | good, race).cumsum()

In [22]:
ProbScoreEqualsXGivenGoodAndRace = pd.read_csv('ProbScoreEqualsXGivenGoodAndRace.csv')
ProbScoreEqualsXGivenGoodAndRace.head()

,TransRisk Score,asian,black,hispanic,white
0,0.0,0.000000,0.000000,0.000000,0.000000
1,0.5,0.000000,0.000161,0.000172,0.000000
2,1.0,0.000568,0.002871,0.000355,0.000304
3,1.5,0.000000,0.000669,0.000000,0.000097
4,2.0,0.000327,0.000924,0.000346,0.000205


In [24]:
ProbScoreEqualsXGivenGoodAndRace.cumsum().head()

,TransRisk Score,asian,black,hispanic,white
0,0.0,0.000000,0.000000,0.000000,0.000000
1,0.5,0.000000,0.000161,0.000172,0.000000
2,1.5,0.000568,0.003032,0.000527,0.000304
3,3.0,0.000568,0.003701,0.000527,0.000401
4,5.0,0.000895,0.004625,0.000874,0.000606
